In [1]:
# Install dependencies (run this cell first!)
!uv pip install -q "pydantic-ai-slim[logfire,openai,tavily,a2a]>=0.0.49" "pydantic-settings>=2.0.0" "logfire[httpx]>=4.3.3" "httpx>=0.27.0"
print("✅ Dependencies installed successfully!")

✅ Dependencies installed successfully!


# Lecture 03: Agent-as-Tool Collaboration Pattern

This notebook demonstrates the **Agent-as-Tool** collaboration paradigm, where one agent uses another specialized agent as a tool. The financial assistant from Lecture 01 is enhanced with a `stock_report` tool that internally delegates to the Lecture 02 stock analysis agent.

## How Agent-as-Tool Works: Enhanced ReAct Pattern

This extends the ReAct pattern from Lecture 01 by adding multiple tool options. The agent now chooses between different tools based on the type of information needed:

```mermaid
graph TD
    Start([👤 User Question]) --> Reason[🤖 Agent: Analyze & Reason<br/>- Understand question<br/>- Plan approach<br/>- Decide on tools]
    Reason --> Decision{Need<br/>Tools?}
    
    Decision -->|No| Response[📝 Generate<br/>Final Response]
    Decision -->|Yes| ToolChoice{Which<br/>Tool?}
    
    ToolChoice -->|General Info| WebSearch[🔧 Web Search<br/>Tool]
    ToolChoice -->|Stock Analysis| StockReport[🔧 Stock Report<br/>Tool<br/>Agent-as-Tool]
    
    WebSearch --> Results[📊 Receive<br/>Tool Results]
    StockReport -->|Delegates to<br/>Lecture 02 Agent| StockResults[📊 Receive<br/>StockReport Object]
    StockResults --> Results
    
    Results --> React[🤖 Agent: React<br/>- Process results<br/>- Integrate information<br/>- Decide next step]
    
    React --> Decision
    
    Response --> End([✅ Return Answer<br/>to User])
    
    style Start fill:#e1f5ff,stroke:#01579b,stroke-width:3px
    style Reason fill:#fff9c4,stroke:#f57f17,stroke-width:2px
    style Decision fill:#f3e5f5,stroke:#4a148c,stroke-width:2px
    style ToolChoice fill:#ffebee,stroke:#b71c1c,stroke-width:2px
    style WebSearch fill:#e8f5e9,stroke:#1b5e20,stroke-width:2px
    style StockReport fill:#ffebee,stroke:#b71c1c,stroke-width:2px
    style Results fill:#e8f5e9,stroke:#1b5e20,stroke-width:2px
    style StockResults fill:#ffebee,stroke:#b71c1c,stroke-width:2px
    style React fill:#fff9c4,stroke:#f57f17,stroke-width:2px
    style Response fill:#e0f2f1,stroke:#004d40,stroke-width:2px
    style End fill:#e1f5ff,stroke:#01579b,stroke-width:3px
```

**Key Differences from Lecture 01:**
- **Multiple Tools**: Agent now has 2 tools to choose from (web_search and stock_report)
- **Tool Selection**: Additional decision node for choosing the appropriate tool
- **Agent-as-Tool**: The stock_report tool is actually another agent (from Lecture 02) that runs independently
- **Structured Output**: Stock analysis returns typed `StockReport` objects instead of raw text

**Key Benefits:**
- **Modularity**: Specialized agents can be reused as tools by other agents
- **Separation of Concerns**: Each agent focuses on its specific domain
- **Type Safety**: Agents return structured Pydantic models ensuring data consistency
- **Reusability**: The stock analysis agent can be used standalone or as a tool


In [ ]:
# Setup: Import necessary modules
from pydantic_ai import Agent, RunContext
from pydantic_ai.tools import Tool
from common.models import StockReport
from common.tools import web_search_tool
from common.utils import create_agent_model, setup_logging
from lecture02.agent import analyze_stock

# Initialize logging
setup_logging()

In [ ]:
# Add Logfire setup
from common.utils import setup_logfire

setup_logfire(
    service_name="tool-delegation-lecture03",
    start_message="🚀 Lecture 03 - Tool Delegation Notebook Started",
)

In [ ]:
# Create a tool that delegates to the lecture02 stock analysis agent
async def stock_report(ctx: RunContext, symbol: str) -> StockReport:
    """Get a detailed stock analysis report by delegating to lecture02 agent."""
    return await analyze_stock(symbol)


stock_report_tool = Tool(
    stock_report,
    description="Get a detailed stock analysis report for a stock symbol (e.g., AAPL, TSLA).",
)

# Create enhanced financial assistant with both web_search and stock_report tools
enhanced_financial_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool, stock_report_tool],  # Now has 2 tools
    system_prompt="""
You are a knowledgeable financial assistant.

When answering questions:
1. Use the web_search tool for general financial information
2. Use the stock_report tool when users ask for detailed analysis of specific stocks
3. Combine information from both sources with your financial knowledge
4. Provide clear, actionable advice
5. Include appropriate disclaimers for investment advice

Keep your responses informative but concise.
""",
)

In [ ]:
# Ask a general financial question (uses web_search)
question = "What is the current price of Bitcoin?"
result = await enhanced_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")

In [ ]:
# Ask for detailed stock analysis (uses stock_report tool)
question = "Can you give me a detailed analysis of Apple stock (AAPL)?"
result = await enhanced_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")

In [ ]:
# Try another stock analysis
question = "Give me a comprehensive report on Tesla (TSLA)"
result = await enhanced_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")